# Tutorial #2:  Deploy a classification model in Azure Container Instance (ACI)

In the [previous tutorial](predict-employee-retention-part1-training.ipynb), you trained machine learning models and then registered a model in your workspace on the cloud.  

Now, you're ready to deploy the model as a web service in [Azure Container Instances](https://docs.microsoft.com/azure/container-instances/) (ACI). The web service here is a Docker image that encapsulates the scoring logic and the model itself. 

In this part of the tutorial, you use Azure Machine Learning service to:

* Set up your testing environment
* Retrieve the model from your workspace
* Test the model locally
* Deploy the model to ACI
* Test the deployed model

ACI is a great solution for testing and understanding the workflow. For scalable production deployments, consider using Azure Kubernetes Service. For more information, see [how to deploy and where](https://docs.microsoft.com/azure/machine-learning/service/how-to-deploy-and-where).



# Connect to workspace

Create a workspace object from the existing workspace. `Workspace.from_config()` reads the file **config.json** and loads the details into an object named `workspace`.

If you see this message:
"Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code &lt;token\&gt; to authenticate."
    
Click on the link and use the &lt;token\&gt; given to authenticate. After authenticated, run this script again to get load the Workspace.&lt;/token\&gt;&lt;/token\&gt;

In [ ]:
# Load workspace configuration from the config.json file in the current folder.
from azureml.core import Workspace
workspace = Workspace.from_config()
print(workspace.name, workspace.location, workspace.resource_group, workspace.location, sep='\t')

### Import Azure Machine Learning SDK for Python 

This step is to test you have installed Azure Machine Learning SDK for Python. Most of the coding will required the use of the Azure ML SDK. 

Display the Azure Machine Learning SDK version.

In [ ]:
import azureml.core

# check core SDK version number (need Python 3.6 kernel if you run this in Microsoft Azure Notebooks)
print("Azure ML SDK Version: ", azureml.core.VERSION)

## Test model locally

Before deploying, make sure your model is working locally by:
* Loading test data
* Predicting test data
* Examining the confusion matrix

### Load test data

Load the test data from Datastore. 
You can create your test data, but for simplicity this tutorial will only re-use the same dataset from Tutorial #1.

In [ ]:
from azureml.core import Dataset
import pandas as pd

tabular_dataset = Dataset.get_by_name(workspace, name='predict-employee-retention-tabular').to_pandas_dataframe()

# Process data...
tabular_dataset = tabular_dataset.rename(columns={"sales": "department"})
salary_map = {"low": 0, "medium": 1, "high": 2}
tabular_dataset["salary"] = tabular_dataset["salary"].map(salary_map)
tabular_dataset = pd.get_dummies(tabular_dataset, columns=["department"], drop_first=True)
display(tabular_dataset.describe())

# Testing data
X_test = tabular_dataset.loc[:, tabular_dataset.columns != "left"].values
y_test = tabular_dataset.loc[:, tabular_dataset.columns == "left"].values.flatten()

### Retrieve the model

You registered a model in your workspace in the previous tutorial. Now, load this workspace and download the model to your local directory.

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model
import os 
workspace = Workspace.from_config()
model=Model(workspace, 'predict-employee-retention-model') # Default will get the latest version.

model.download(target_dir=os.getcwd(), exist_ok=True)
print(model)

# Get the model file path.
file_path = os.path.join(os.getcwd(), "predict-employee-retention-model.pkl")


### Predict test data

Feed the test dataset to the model to get predictions.

1. For local testing, use "import joblib".

2. For the cloud compute, use "from sklearn.externals import joblib". To workaround the error "ModuleNotFoundError: No module named 'sklearn.linear_model._logistic'", need to use joblib from the older version of scikit-learn 0.20.3.


In [ ]:
import joblib # Use this when retrieve model created on local

In [ ]:
from sklearn.externals import joblib # Use this when retrieve model created on cloud.

In [ ]:
clf = joblib.load(file_path)
y_hat = clf.predict(X_test)

# Quick test to determine the model is working! 
print(y_hat.shape) # you should see (14999,)

## Deploy as web service

Once you've tested the model and are satisfied with the results, deploy the model as a web service hosted in ACI. 

To build the correct environment for ACI, provide the following:
* A scoring script to show how to use the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before

Note: the deployed web service can be found in your Workspace &gt; Deployments.

### Create scoring script

Create the scoring script, called score.py, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:
* The `init()` function, which typically loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.


In [ ]:
%%writefile score.py
import json
import numpy as np
import os
import pickle

from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('predict-employee-retention-model')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

### Deploy in ACI
Configure the image and deploy. The following code goes through these steps:

1. Build an image using:
   * The scoring file (`score.py`)
   * The environment and resources required
   * The model file
1. Register that image under the workspace. 
1. Send the image to the ACI container.
1. Start up a container in ACI using the image.
1. Get the web service HTTP endpoint.

The image can be found in your Azure ML Workspace &gt; Images.

Note:
If you see "ERROR - Error, there is already a service with name sklearn-employee-retention-svc found in workspace <your workspace name>", 
go to your **Azure ML Workspace &gt; Deployments**, where you can delete it if you need to recreate the service.


This step may take a while to start after you run the cell, you will see the message "Running" appearing when it starts and will take few minutes to complete.

In [ ]:
%%time
from azureml.core import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.conda_dependencies import CondaDependencies

# Configure the environment to run the scoring script.
env = Environment('my_env')
cd = CondaDependencies.create(pip_packages=['azureml-sdk','azureml-defaults','scikit-learn==0.20.3'])
env.python.conda_dependencies = cd

# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# Set deployment configuration. While it depends on your model, the default of 1 core and 1 gigabyte of RAM 
# is usually sufficient for many models. If you feel you need more later, you would have to recreate the 
# image and redeploy the service.
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Employee retention",  "method" : "sklearn"}, 
                                               description='Predict employee retention with sklearn')

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(
    workspace = workspace,
    name = "sklearn-employee-retention-svc",
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config)

service.wait_for_deployment(show_output=True)

Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [ ]:
service = Webservice(workspace=workspace, name='sklearn-employee-retention-svc')
print(service.scoring_uri)

## Test deployed service

Test the deployed model with a random sample of 30 test data. For simplicity, the same
training data is used.

The following code goes through these steps:
1. Send the data as a JSON array to the web service hosted in ACI. 

1. Use the SDK's `run` API to invoke the service. You can also make raw calls using any HTTP tool such as curl.

1. Print the returned predictions. Red font is used to highlight the misclassified samples. 

1 = predict the employee will leave the organization

0 = predict the employee will stay

Run below code cell few times to see different predictions.

Note: if you see the message "Figure size 2000x100 with 30 Axes" and nothing is displayed, re-run the code cell again.

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

# find 30 random samples from test set
n = 30
sample_indices = np.random.permutation(X_test.shape[0])[0:n]

test_samples = json.dumps({"data": X_test[sample_indices].tolist()})
test_samples = bytes(test_samples, encoding='utf8')

# predict using the deployed model
result = service.run(input_data=test_samples)

# compare actual value vs. the predicted values:
i = 0
plt.figure(figsize = (20, 1))

for s in sample_indices:
    plt.subplot(1, n, i + 1)
    plt.axhline('')
    plt.axvline('')
    
    # use different color for misclassified sample
    font_color = 'red' if y_test[s] != result[i] else 'black'
        
    plt.text(x=0, y=-5, s=result[i], fontsize=18, color=font_color)
    plt.imshow(X_test[s].reshape(1, 17))
    
    i = i + 1
plt.show()

You can also send raw HTTP request to test the web service. Run below code cell few times to see different predictions.

In [ ]:
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(X_test)-1)
input_data = "{\"data\": [" + str(list(X_test[random_index])) + "]}"

headers = {'Content-Type':'application/json'}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("input data:", input_data)
print("label:", y_test[random_index])
print("prediction:", resp.text)

## Clean up resources

To keep the resource group and workspace for other tutorials and exploration, you can delete only the ACI deployment using this API call:

In [ ]:
service.delete()

You can also manually delete the deployed web service which can be found in your **Azure ML Workspace &gt; Deployments**.

If you're not going to use what you've created here, delete the resources you just created so you don't incur any charges. 